<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/qft_tensornetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tensor Networks for Quantum Fourier Transform with Cirq**

https://github.com/NVIDIA/cuQuantum/blob/main/python/samples/tensornet/experimental/network_state/circuits_cirq/example07_mpi_sampling.py

https://docs.nvidia.com/cuda/cuquantum/latest/python/tensornet.html#tn-simulator-intro

In [ ]:
!pip install cirq -q

In [ ]:
!pip install mpi4py -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install cuquantum -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.3/202.3 MB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install cupy-cuda11x -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 MB 7.0 MB/s eta 0:00:00


In [ ]:
# Copyright (c) 2024, NVIDIA CORPORATION & AFFILIATES
#
# SPDX-License-Identifier: BSD-3-Clause

import cirq

import cupy as cp
from mpi4py import MPI

from cuquantum.bindings import cutensornet as cutn
from cuquantum.tensornet.experimental import NetworkState, TNConfig

root = 0
comm = MPI.COMM_WORLD
rank, size = comm.Get_rank(), comm.Get_size()
if rank == root:
    print("*** Printing is done only from the root process to prevent jumbled messages ***")
    print(f"The number of processes is {size}")

num_devices = cp.cuda.runtime.getDeviceCount()
device_id = rank % num_devices
dev = cp.cuda.Device(device_id)
dev.use()

props = cp.cuda.runtime.getDeviceProperties(dev.id)
if rank == root:
    print("cuTensorNet-vers:", cutn.get_version())
    print("===== root process device info ======")
    print("GPU-name:", props["name"].decode())
    print("GPU-clock:", props["clockRate"])
    print("GPU-memoryClock:", props["memoryClockRate"])
    print("GPU-nSM:", props["multiProcessorCount"])
    print("GPU-major:", props["major"])
    print("GPU-minor:", props["minor"])
    print("========================")

handle = cutn.create()
cutn_comm = comm.Dup()
cutn.distributed_reset_configuration(handle, MPI._addressof(cutn_comm), MPI._sizeof(cutn_comm))
if rank == root:
    print("Reset distributed MPI configuration")

free_mem = dev.mem_info[0]
free_mem = comm.allreduce(free_mem, MPI.MIN)
workspace_limit = int(free_mem * 0.5)

# device id must be explicitly set on each process
options = {'handle': handle,
           'device_id': device_id,
           'memory_limit': workspace_limit}

# create a QFT circuit
n_qubits = 12
qubits = cirq.LineQubit.range(n_qubits)
qft_operation = cirq.qft(*qubits, without_reverse=True)
circuit = cirq.Circuit(qft_operation)
if rank == root:
    print(circuit)

# select tensor network contraction as the simulation method
config = TNConfig(num_hyper_samples=4)

# create a NetworkState object
with NetworkState.from_circuit(circuit, dtype='complex128', backend='cupy', config=config, options=options) as state:
    # draw samples from the state object
    nshots = 1000
    samples = state.compute_sampling(nshots)
    if rank == root:
        print("Sampling results:")
        print(samples)

cutn.destroy(handle)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

This Python code snippet demonstrates how to simulate a Quantum Fourier Transform (QFT) circuit using cuQuantum's `tensornet` library in a distributed, multi-GPU environment using MPI (Message Passing Interface). Let's break down the code step by step:

**1. Importing Libraries:**

```python
import cirq
import cupy as cp
from mpi4py import MPI
from cuquantum.bindings import cutensornet as cutn
from cuquantum.tensornet.experimental import NetworkState, TNConfig
```

* `cirq`: A Python library for creating, manipulating, and simulating quantum circuits.
* `cupy`: A NumPy-compatible array library for GPU acceleration.
* `mpi4py`: A Python interface to the MPI standard for parallel computing.
* `cuquantum.bindings.cutensornet`: The cuTensorNet library bindings for tensor network computations.
* `cuquantum.tensornet.experimental.NetworkState`, `TNConfig`: Classes for managing and configuring tensor network simulations.

**2. MPI Initialization:**

```python
root = 0
comm = MPI.COMM_WORLD
rank, size = comm.Get_rank(), comm.Get_size()
if rank == root:
    print("*** Printing is done only from the root process to prevent jumbled messages ***")
    print(f"The number of processes is {size}")
```

* This section initializes MPI.
* `comm` represents the communicator, which allows processes to communicate with each other.
* `rank` is the unique ID of each process, and `size` is the total number of processes.
* The `if rank == root:` block ensures that output is printed only by the root process (rank 0) to avoid messy output.

**3. GPU Device Selection:**

```python
num_devices = cp.cuda.runtime.getDeviceCount()
device_id = rank % num_devices
dev = cp.cuda.Device(device_id)
dev.use()

props = cp.cuda.runtime.getDeviceProperties(dev.id)
if rank == root:
    print("cuTensorNet-vers:", cutn.get_version())
    print("===== root process device info ======")
    print("GPU-name:", props["name"].decode())
    print("GPU-clock:", props["clockRate"])
    print("GPU-memoryClock:", props["memoryClockRate"])
    print("GPU-nSM:", props["multiProcessorCount"])
    print("GPU-major:", props["major"])
    print("GPU-minor:", props["minor"])
    print("========================")
```

* This part selects a GPU for each process.
* It calculates the `device_id` by taking the remainder of the process rank divided by the number of available GPUs.
* `dev.use()` sets the selected GPU as the current device for the process.
* It then prints GPU information on the root process.

**4. cuTensorNet Initialization and MPI Configuration:**

```python
handle = cutn.create()
cutn_comm = comm.Dup()
cutn.distributed_reset_configuration(handle, MPI._addressof(cutn_comm), MPI._sizeof(cutn_comm))
if rank == root:
    print("Reset distributed MPI configuration")
```

* This initializes the cuTensorNet library and configures it for distributed execution using MPI.
* `cutn.create()` creates a cuTensorNet handle.
* `cutn.distributed_reset_configuration()` sets up the library to work with the MPI communicator.

**5. Workspace Memory Allocation:**

```python
free_mem = dev.mem_info[0]
free_mem = comm.allreduce(free_mem, MPI.MIN)
workspace_limit = int(free_mem * 0.5)

options = {'handle': handle,
            'device_id': device_id,
            'memory_limit': workspace_limit}
```

* This section determines the available GPU memory and sets a memory limit for the cuTensorNet workspace.
* `comm.allreduce()` finds the minimum available memory across all processes.
* The `options` dictionary stores configuration parameters for cuTensorNet.

**6. Creating the QFT Circuit:**

```python
n_qubits = 12
qubits = cirq.LineQubit.range(n_qubits)
qft_operation = cirq.qft(*qubits, without_reverse=True)
circuit = cirq.Circuit(qft_operation)
if rank == root:
    print(circuit)
```

* This creates a 12-qubit QFT circuit using Cirq.

**7. Tensor Network Simulation and Sampling:**

```python
config = TNConfig(num_hyper_samples=4)

with NetworkState.from_circuit(circuit, dtype='complex128', backend='cupy', config=config, options=options) as state:
    nshots = 1000
    samples = state.compute_sampling(nshots)
    if rank == root:
        print("Sampling results:")
        print(samples)
```

* This is the core of the simulation.
* `TNConfig` configures the tensor network contraction.
* `NetworkState.from_circuit()` creates a tensor network representation of the circuit.
* `state.compute_sampling()` performs the sampling and returns the results.
* The results are printed on the root process.

**8. cuTensorNet Destruction:**

```python
cutn.destroy(handle)
```

* This releases the resources used by the cuTensorNet handle.

**In summary:**

This code leverages cuQuantum's cuTensorNet library for efficient, distributed simulation of quantum circuits on GPUs. It uses MPI to distribute the computational workload across multiple GPUs, allowing for the simulation of larger quantum systems. It creates a QFT circuit using Cirq and then samples from the output distribution of that circuit using cuTensorNet's tensor network capabilities.
